# Gettting our DataFrames

In [191]:
import pandas as pd
import numpy as np
import collections
import sagemaker
from sagemaker import get_execution_role
import boto3
import io
import os
import sagemaker.amazon.common as smac
from sagemaker.serializers import CSVSerializer
import ast

In [71]:
# Regular Season Results since 1985 (Only includes who won and the points)
reg_season_compact_pd = pd.read_csv('MRegularSeasonCompactResults.csv') 

# Regular Season Results since 2003 but includes useful stats like Rebounds, Assists, etc.
reg_season_detailed_pd = pd.read_csv('MRegularSeasonDetailedResults.csv')

# List of teams who are/was in Division I along with their ID
teams_pd = pd.read_csv('MTeams.csv')

# Like the first Dataframe but for the tournament (We will not use the detailed one)
tourney_compact_pd = pd.read_csv('MNCAATourneyCompactResults.csv')

# The Conference Tourney Detailed Results since 2003
conference_tourney_results_pd = pd.read_csv('MConferenceTourneyGames.csv')

# List of Teams along with their Conferences and ID per Year
conferences_pd = pd.read_csv('MTeamConferences.csv')

# Seeds 
seeds_pd = pd.read_csv('MNCAATourneySeeds.csv')

# List of Conference Tourney Winners since 2001
l = []
for i in range(len(conference_tourney_results_pd) - 1):
    if conference_tourney_results_pd.iloc[i, 1] != conference_tourney_results_pd.iloc[i + 1, 1]:
        season = conference_tourney_results_pd.iloc[i, 0]
        conference = conference_tourney_results_pd.iloc[i, 1]
        winner = conference_tourney_results_pd.iloc[i, 3]
        l.append({'Season': season, 'Conference': conference, 'Winner': winner})
        
conference_tourney_winners_pd = pd.DataFrame(l)

# Preparing and Obtaining the Training Set

In [72]:
# Get ID given Name
def getTeamID(name):
    return teams_pd[teams_pd['TeamName'] == name].values[0][0]

# Get Name given ID
def getTeamName(team_id):
    return teams_pd[teams_pd['TeamID'] == team_id].values[0][1]

In [73]:
# How many wins did a Team win in a given Season
def getRegSeasonWins(team_id, year):
    c1 = reg_season_compact_pd['WTeamID'] == team_id
    c2 = reg_season_compact_pd['Season'] == year
    return len(reg_season_compact_pd[c1 & c2])

# What was a team's Points per Game in a given Season
def getPPG(team_id, year):
    ppg = 0
    c1 = reg_season_compact_pd['WTeamID'] == team_id
    c2 = reg_season_compact_pd['Season'] == year
    c3 = reg_season_compact_pd['LTeamID'] == team_id
    gamesWon = reg_season_compact_pd[c1 & c2]
    ppg = gamesWon['WScore'].sum()
    gamesLost = reg_season_compact_pd[c2 & c3]
    ppg += gamesLost['LScore'].sum()
    total_games = len(gamesWon) + len(gamesLost)
    ppg /= total_games
    return round(ppg, 2)

# In a given season, how many points did a given team give up per game
def getOPPG(team_id, year):
    oppg = 0
    c1 = reg_season_compact_pd['WTeamID'] == team_id
    c2 = reg_season_compact_pd['Season'] == year
    c3 = reg_season_compact_pd['LTeamID'] == team_id
    gamesWon = reg_season_compact_pd[c1 & c2]
    oppg = gamesWon['LScore'].sum()
    gamesLost = reg_season_compact_pd[c2 & c3]
    oppg += gamesLost['WScore'].sum()
    total_games = len(gamesWon) + len(gamesLost)
    oppg /= total_games
    return round(oppg, 2)

In [74]:
# Set conditions for Power 6 Conference
acc = conferences_pd['ConfAbbrev'] == 'acc'
big12 = conferences_pd['ConfAbbrev'] == 'big_twelve'
bigeast = conferences_pd['ConfAbbrev'] == 'big_east'
big10 = conferences_pd['ConfAbbrev'] == 'big_ten'
pac12 = conferences_pd['ConfAbbrev'] == 'pac_twelve'
sec = conferences_pd['ConfAbbrev'] == 'sec'

# Make Dataframe where it only contains Power 6 teams
p6 = conferences_pd[acc | big12 | bigeast | big10 | pac12 | sec]

def getPower6(team_id, year):
    # Filter out the Dataframe for the appropiate year
    c1 = conferences_pd['Season'] == year
    
    # Get the list of TeamID's that sufficies all the conditions
    p6_list = list(p6[c1]['TeamID'])

    if team_id in p6_list:
        return 1
    else:
        return 0

In [75]:
# How many three's did a team make per game in a given season
def get3PT(team_id, year):
    if year < 2003:
        return 0
    threes = 0
    c1 = reg_season_detailed_pd['WTeamID'] == team_id
    c2 = reg_season_detailed_pd['Season'] == year
    c3 = reg_season_detailed_pd['LTeamID'] == team_id
    gamesWon = reg_season_detailed_pd[c1 & c2]
    threes = gamesWon['WFGM3'].sum()
    gamesLost = reg_season_detailed_pd[c2 & c3]
    threes += gamesLost['LFGM3'].sum()
    total_games = len(gamesWon) + len(gamesLost)
    threes /= total_games
    return round(threes, 2)

# How many turnovers did a team make per game in a given season
def getTO(team_id, year):
    if year < 2003:
        return 0
    to = 0
    c1 = reg_season_detailed_pd['WTeamID'] == team_id
    c2 = reg_season_detailed_pd['Season'] == year
    c3 = reg_season_detailed_pd['LTeamID'] == team_id
    gamesWon = reg_season_detailed_pd[c1 & c2]
    to = gamesWon['WTO'].sum()
    gamesLost = reg_season_detailed_pd[c2 & c3]
    to += gamesLost['LTO'].sum()
    total_games = len(gamesWon) + len(gamesLost)
    to /= total_games
    return round(to, 2)

In [76]:
# How many Assists did a team make per game
def getAST(team_id, year):
    if year < 2003:
        return 0
    ast = 0
    c1 = reg_season_detailed_pd['WTeamID'] == team_id
    c2 = reg_season_detailed_pd['Season'] == year
    c3 = reg_season_detailed_pd['LTeamID'] == team_id
    gamesWon = reg_season_detailed_pd[c1 & c2]
    ast = gamesWon['WAst'].sum()
    gamesLost = reg_season_detailed_pd[c2 & c3]
    ast += gamesLost['LAst'].sum()
    total_games = len(gamesWon) + len(gamesLost)
    ast /= total_games
    return round(ast, 2)

# Determine if a team was the conference champion in their division in a given year
def getTourneyConferenceChampion(team_id, year):
    if year < 2001:
        return 0
    conf = getConference(team_id, year)
    c1 = conference_tourney_winners_pd['Season'] == year
    c2 = conference_tourney_winners_pd['Conference'] == conf
    if len(conference_tourney_winners_pd[c1 & c2]) == 0:
        return 0
    if team_id == conference_tourney_winners_pd[c1 & c2]['Winner'].values[0]:
        return 1
    else:
        return 0

In [77]:
# Easy way to get a team's conference in a given year
def getConference(team_id, year):
    c1 = conferences_pd['TeamID'] == team_id
    c2 = conferences_pd['Season'] == year
    c3 = conferences_pd[c1 & c2]
    if len(c3) == 0:
        return conferences_pd[c1].values[0][2]
    return c3['ConfAbbrev'].values[0]

# Get the seed of the team in a given year
def getSeed(team_id, year):
    c1 = seeds_pd['TeamID'] == team_id
    c2 = seeds_pd['Season'] == year
    if len(seeds_pd[c1 & c2]) == 0:
        return 0
    return int(seeds_pd[c1 & c2]['Seed'].values[0][1:3])

# Get rebounds per game in a given year
def getRPG(team_id, year):
    if year < 2003:
        return 0
    reb = 0
    c1 = reg_season_detailed_pd['WTeamID'] == team_id
    c2 = reg_season_detailed_pd['Season'] == year
    c3 = reg_season_detailed_pd['LTeamID'] == team_id
    gamesWon = reg_season_detailed_pd[c1 & c2]
    reb = gamesWon['WOR'].sum()
    reb += gamesWon['WDR'].sum()
    gamesLost = reg_season_detailed_pd[c2 & c3]
    reb += gamesLost['LOR'].sum()
    reb += gamesLost['LDR'].sum()
    total_games = len(gamesWon) + len(gamesLost)
    reb /= total_games
    return round(reb, 2)

In [78]:
# Steals per game
def getSTL(team_id, year):
    if year < 2003:
        return 0
    stl = 0
    c1 = reg_season_detailed_pd['WTeamID'] == team_id
    c2 = reg_season_detailed_pd['Season'] == year
    c3 = reg_season_detailed_pd['LTeamID'] == team_id
    gamesWon = reg_season_detailed_pd[c1 & c2]
    stl = gamesWon['WStl'].sum()
    gamesLost = reg_season_detailed_pd[c2 & c3]
    stl += gamesLost['LStl'].sum()
    total_games = len(gamesWon) + len(gamesLost)
    stl /= total_games
    return round(stl, 2)

# How many times did a team appear in the tournament as of 2022
def getNumOfAppearances(team_id):
    return len(seeds_pd[seeds_pd['TeamID'] == team_id])

# Helper function for below
def getHomeStat(row):
    if (row == 'H'):
        home = 1
    if (row == 'A'):
        home = -1
    if (row == 'N'):
        home = 0
    return home

In [79]:
# Build the vector
def getSeasonData(team_id, year):
    # Check first if the team was Division 1 at the time
    c1 = teams_pd[teams_pd['TeamID'] == team_id]['FirstD1Season'].values[0] <= year
    c2 = teams_pd[teams_pd['TeamID'] == team_id]['LastD1Season'].values[0] >= year
    if ~c1 or ~c2:
        return []
    return [getRegSeasonWins(team_id, year),
            getPPG(team_id, year),
            getOPPG(team_id, year),
            getPower6(team_id, year),
            get3PT(team_id, year),
            getTO(team_id, year),
            getAST(team_id, year),
            getTourneyConferenceChampion(team_id, year),
            getSeed(team_id, year),
            getRPG(team_id, year),
            getSTL(team_id, year),
            getNumOfAppearances(team_id)]

# Build vectors for every team in a given season
def createSeasonDict(year):
    seasonDictionary = collections.defaultdict(list)
    for team in teams_pd['TeamName'].tolist():
        team_id = teams_pd[teams_pd['TeamName'] == team].values[0][0]
        team_vector = getSeasonData(team_id, year)
        seasonDictionary[team_id] = team_vector
    return seasonDictionary

In [80]:
# Basically run the function directly above this one on a set of years
def createTrainingSet(years):
    totalNumGames = 0
    for year in years:
        season = reg_season_compact_pd[reg_season_compact_pd['Season'] == year]
        totalNumGames += len(season.index)
        tourney = tourney_compact_pd[tourney_compact_pd['Season'] == year]
        totalNumGames += len(tourney.index)
    numFeatures = len(getSeasonData(1181,2012)) #Just choosing a random team and seeing the dimensionality of the vector
    xTrain = np.zeros(( totalNumGames, numFeatures + 1))
    yTrain = np.zeros(( totalNumGames ))
    indexCounter = 0
    for year in years:
        team_vectors = createSeasonDict(year)
        season = reg_season_compact_pd[reg_season_compact_pd['Season'] == year]
        numGamesInSeason = len(season.index)
        tourney = tourney_compact_pd[tourney_compact_pd['Season'] == year]
        numGamesInSeason += len(tourney.index)
        xTrainSeason = np.zeros(( numGamesInSeason, numFeatures + 1))
        yTrainSeason = np.zeros(( numGamesInSeason ))
        counter = 0
        for index, row in season.iterrows():
            w_team = row['WTeamID']
            w_vector = team_vectors[w_team]
            l_team = row['LTeamID']
            l_vector = team_vectors[l_team]
            diff = [a - b for a, b in zip(w_vector, l_vector)]
            home = getHomeStat(row['WLoc'])
            if (counter % 2 == 0):
                diff.append(home) 
                xTrainSeason[counter] = diff
                yTrainSeason[counter] = 1
            else:
                diff.append(-home)
                xTrainSeason[counter] = [ -p for p in diff]
                yTrainSeason[counter] = 0
            counter += 1
        for index, row in tourney.iterrows():
            w_team = row['WTeamID']
            w_vector = team_vectors[w_team]
            l_team = row['LTeamID']
            l_vector = team_vectors[l_team]
            diff = [a - b for a, b in zip(w_vector, l_vector)]
            home = 0 #All tournament games are neutral
            if (counter % 2 == 0):
                diff.append(home) 
                xTrainSeason[counter] = diff
                yTrainSeason[counter] = 1
            else:
                diff.append(-home)
                xTrainSeason[counter] = [ -p for p in diff]
                yTrainSeason[counter] = 0
            counter += 1
        xTrain[indexCounter:numGamesInSeason+indexCounter] = xTrainSeason
        yTrain[indexCounter:numGamesInSeason+indexCounter] = yTrainSeason
        indexCounter += numGamesInSeason
    return xTrain, yTrain

In [81]:
# Get training set for 2023
X_train, y_train = createTrainingSet([2023])

/home/ec2-user/anaconda3/envs/amazonei_pytorch_latest_p37/lib/python3.7/site-packages/ipykernel_launcher.py:17: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  app.launch_new_instance()


In [82]:
# Upload the training data to the bucket
X_train = np.array(X_train).astype('float32')
y_train = np.array(y_train).astype('float32')

sess = sagemaker.Session()
bucket = 'mmbracket-29122'
role = get_execution_role()
prefix = 'logistic-regression'
key = 'train-data'

buf = io.BytesIO()
smac.write_numpy_to_dense_tensor(buf, X_train, y_train)
buf.seek(0)
boto3.resource('s3').Bucket(bucket).Object(os.path.join(prefix, 'train', key)).upload_fileobj(buf)

s3_train = 's3://{}/{}/train/{}'.format(bucket, prefix, key)

# Model Training

In [84]:
container = '174872318107.dkr.ecr.us-west-2.amazonaws.com/linear-learner:1'

lr = sagemaker.estimator.Estimator(container,
                                   role,
                                   instance_count = 1,
                                   instance_type = 'ml.m5.4xlarge',
                                   output_path = 's3://{}/{}/output'.format(bucket, prefix),
                                   sagemaker_session = sess)

lr.set_hyperparameters(feature_dim = 13,
                       predictor_type = 'binary_classifier'
                      )

lr.fit({'train': s3_train})

INFO:sagemaker:Creating training-job with name: linear-learner-2023-08-17-21-56-26-979


2023-08-17 21:56:27 Starting - Starting the training job...
2023-08-17 21:56:41 Starting - Preparing the instances for training......
2023-08-17 21:57:46 Downloading - Downloading input data...
2023-08-17 21:58:11 Training - Downloading the training image......
2023-08-17 21:59:02 Training - Training image download completed. Training in progress....
2023-08-17 21:59:52 Uploading - Uploading generated training modelDocker entrypoint called with argument(s): train
Running default environment configuration script
[08/17/2023 21:59:41 INFO 139633801307968] Reading default configuration from /opt/amazon/lib/python3.7/site-packages/algorithm/resources/default-input.json: {'mini_batch_size': '1000', 'epochs': '15', 'feature_dim': 'auto', 'use_bias': 'true', 'binary_classifier_model_selection_criteria': 'accuracy', 'f_beta': '1.0', 'target_recall': '0.8', 'target_precision': '0.8', 'num_models': 'auto', 'num_calibration_samples': '10000000', 'init_method': 'uniform', 'init_scale': '0.07', 'in

In [231]:
lr_predict = lr.deploy(initial_instance_count = 1,
                       instance_type = 'ml.t2.medium',
                       endpoint_name = 'mm-endpoint3',
                       serializer = CSVSerializer()
                      )

INFO:sagemaker:Creating model with name: linear-learner-2023-08-17-23-43-04-498
INFO:sagemaker:Creating endpoint-config with name mm-endpoint3
INFO:sagemaker:Creating endpoint with name mm-endpoint3


---------------!

# Model Testing

In [142]:
# First we will get the actual winners of the 2023 Tournament
playin = [1394, 1338, 1192, 1113]

actual_winners = []

r1 = [1163, 1361, 1274, 1194, 1400, 1166, 1211, 1243,
      1397, 1277, 1417, 1116, 1343, 1462, 1104, 1222,
      1268, 1120, 1202, 1231, 1124, 1338, 1335, 1281,
      1192, 1242, 1181, 1388, 1246, 1395, 1321, 1266]

r2 = [1163, 1361, 1274, 1194, 1400, 1166, 1211, 1243,
      1397, 1277, 1417, 1116, 1343, 1462, 1104, 1222]

r3 = [1163, 1361, 1274, 1194, 1400, 1166, 1211, 1243]

r4 = [1163, 1361, 1274, 1194]

r5 = [1163, 1361]

r6 = [1163]

actual_winners.append(r1)
actual_winners.append(r2)
actual_winners.append(r3)
actual_winners.append(r4)
actual_winners.append(r5)
actual_winners.append(r6)

In [227]:
# Next we will get our testing data which are the first round matchups
def swap_it(m, x1, x2):
    m[x1], m[x2] = m[x2], m[x1]
    return m

seeds_2023 = seeds_pd[seeds_pd['Season'] == 2023]

matchups = []
k = 0

for i in range(8):
    if len(seeds_2023.iloc[16 - i, 1]) == 4:
        if seeds_2023.iloc[16 - i, 2] not in playin:
            k = 1
    matchups.append({'Round': 1, 'Team1ID': seeds_2023.iloc[i, 2], 'Team2ID': seeds_2023.iloc[16 - i - k, 2]})
       
k = 0

for i in range(17, 25):
    if len(seeds_2023.iloc[50 - i, 1]) == 4:
        if seeds_2023.iloc[50 - i, 2] not in playin:
            k = 1
    matchups.append({'Round': 1, 'Team1ID': seeds_2023.iloc[i, 2], 'Team2ID': seeds_2023.iloc[50 - i - k, 2]})
    
k = 0

for i in range(34, 42):
    if len(seeds_2023.iloc[84 - i, 1]) == 4:
        if seeds_2023.iloc[84 - i, 2] not in playin:
            k = 1
    matchups.append({'Round': 1, 'Team1ID': seeds_2023.iloc[i, 2], 'Team2ID': seeds_2023.iloc[84 - i - k, 2]})

k = 0
    
for i in range(51, 59):
    if len(seeds_2023.iloc[118 - i, 1]) == 4:
        if seeds_2023.iloc[118 - i, 2] not in playin:
            k = 1
    matchups.append({'Round': 1, 'Team1ID': seeds_2023.iloc[i, 2], 'Team2ID': seeds_2023.iloc[118 - i - k, 2]})

for i in range(0, 4):
    matchups = swap_it(matchups, 8 * i + 1, 8 * i + 7)
    matchups = swap_it(matchups, 8 * i + 2, 8 * i + 4)

In [228]:
# Function to predict probability that Team 1 wins
def predictOutcome(team_data1, team_data2):
    diff = [a - b for a, b in zip(team_data1, team_data2)]
    diff.append(0)
    return lr_predict.predict(diff)

In [229]:
# Predict the tournament
r = 2
threshold = 32

# store the odd team
odd_team = 0

for i in range(63):
    # First get the IDs
    team1 = matchups[i]['Team1ID']
    team2 = matchups[i]['Team2ID']
    
    # Then get their data
    t1_data = getSeasonData(team1, 2023)
    t2_data = getSeasonData(team2, 2023)
    
    # Get 0 or 1 value by running the above function
    prob = predictOutcome(t1_data, t2_data)
    prob = ast.literal_eval(prob.decode("UTF-8"))['predictions'][0]
    
    # Predict team
    if prob['predicted_label']: 
        matchups[i]['Predicted_Winner'] = team1
        matchups[i]['Probability'] = prob['score']
    else:
        matchups[i]['Predicted_Winner'] = team2
        matchups[i]['Probability'] = 1 - prob['score']
        
    # Add a new row to the matchups once two games are complete until we have 63 games
    if len(matchups) < 64:
        if i % 2 == 1:
            matchups.append({'Round': r, 'Team1ID': odd_team, 'Team2ID': matchups[i]['Predicted_Winner']})
            odd_team = 0
        else:
            odd_team = team1
            
    if i == threshold:
        threshold += (32 / 2**(r - 1))
        r += 1
        
#matchups

/home/ec2-user/anaconda3/envs/amazonei_pytorch_latest_p37/lib/python3.7/site-packages/ipykernel_launcher.py:17: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  app.launch_new_instance()


[{'Round': 1,
  'Team1ID': 1345,
  'Team2ID': 1192,
  'Predicted_Winner': 1345,
  'Probability': 0.9568223357200623},
 {'Round': 1,
  'Team1ID': 1272,
  'Team2ID': 1194,
  'Predicted_Winner': 1194,
  'Probability': 0.5614498257637024},
 {'Round': 1,
  'Team1ID': 1181,
  'Team2ID': 1331,
  'Predicted_Winner': 1181,
  'Probability': 0.8091337084770203},
 {'Round': 1,
  'Team1ID': 1397,
  'Team2ID': 1418,
  'Predicted_Winner': 1397,
  'Probability': 0.8806710839271545},
 {'Round': 1,
  'Team1ID': 1243,
  'Team2ID': 1286,
  'Predicted_Winner': 1243,
  'Probability': 0.8549213409423828},
 {'Round': 1,
  'Team1ID': 1246,
  'Team2ID': 1344,
  'Predicted_Winner': 1246,
  'Probability': 0.5743269324302673},
 {'Round': 1,
  'Team1ID': 1277,
  'Team2ID': 1425,
  'Predicted_Winner': 1425,
  'Probability': 0.5620112419128418},
 {'Round': 1,
  'Team1ID': 1266,
  'Team2ID': 1436,
  'Predicted_Winner': 1266,
  'Probability': 0.9119142293930054},
 {'Round': 1,
  'Team1ID': 1104,
  'Team2ID': 1394,
  'P

In [236]:
lr_predict.delete_endpoint()

INFO:sagemaker:Deleting endpoint configuration with name: mm-endpoint3
INFO:sagemaker:Deleting endpoint with name: mm-endpoint3
